# Lab 3: Extending Logistic Regression

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

## Dataset Preparation

In [38]:
df = pd.read_csv('Diabetes_Dataset/diabetes_012_health_indicators_BRFSS2015.csv')
print(df.head())
print(df.columns)

   Diabetes_012  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
0           0.0     1.0       1.0        1.0  40.0     1.0     0.0   
1           0.0     0.0       0.0        0.0  25.0     1.0     0.0   
2           0.0     1.0       1.0        1.0  28.0     0.0     0.0   
3           0.0     1.0       0.0        1.0  27.0     0.0     0.0   
4           0.0     1.0       1.0        1.0  24.0     0.0     0.0   

   HeartDiseaseorAttack  PhysActivity  Fruits  ...  AnyHealthcare  \
0                   0.0           0.0     0.0  ...            1.0   
1                   0.0           1.0     0.0  ...            0.0   
2                   0.0           0.0     1.0  ...            1.0   
3                   0.0           1.0     1.0  ...            1.0   
4                   0.0           1.0     1.0  ...            1.0   

   NoDocbcCost  GenHlth  MentHlth  PhysHlth  DiffWalk  Sex   Age  Education  \
0          0.0      5.0      18.0      15.0       1.0  0.0   9.0        4.0   
1     

In [39]:
# Creating target features by modifying first column in original dataframe such that we have 3 features consisting of binary values for
# no diabetes, prediabetes, and diabetes where 0 is False and 1 is True
target_array = np.zeros((len(df),4))
for i in range(len(df)):
    target_array[i,0] = df['Diabetes_012'].values[i]
for i in range(len(target_array)):
    # no diabetes
    if target_array[i,0] == 0:
        target_array[i,1] = 1
    # prediabetes
    if target_array[i,0] == 1:
        target_array[i,2] = 1
    # diabetes
    if target_array[i,0] == 2:
        target_array[i,3] = 1

# Adding new target columns to original dataframe
target_columns = ['NoDiabetes', 'PreDiabetes', 'Diabetes']
for i in range(target_array.shape[1]-1):
    df.insert(i, target_columns[i], target_array[:,1:][:,i], True)
df_target = df.drop('Diabetes_012', axis=1)
print(df_target)

        NoDiabetes  PreDiabetes  Diabetes  HighBP  HighChol  CholCheck   BMI  \
0              1.0          0.0       0.0     1.0       1.0        1.0  40.0   
1              1.0          0.0       0.0     0.0       0.0        0.0  25.0   
2              1.0          0.0       0.0     1.0       1.0        1.0  28.0   
3              1.0          0.0       0.0     1.0       0.0        1.0  27.0   
4              1.0          0.0       0.0     1.0       1.0        1.0  24.0   
...            ...          ...       ...     ...       ...        ...   ...   
253675         1.0          0.0       0.0     1.0       1.0        1.0  45.0   
253676         0.0          0.0       1.0     1.0       1.0        1.0  18.0   
253677         1.0          0.0       0.0     0.0       0.0        1.0  28.0   
253678         1.0          0.0       0.0     1.0       0.0        1.0  23.0   
253679         0.0          0.0       1.0     1.0       1.0        1.0  25.0   

        Smoker  Stroke  HeartDiseaseorA

In [40]:
# Need to remove the target column of diabetes in original dataframe
columns = list(df_target.columns)
targets = ['NoDiabetes', 'PreDiabetes', 'Diabetes']
for col in targets:
    columns.remove(col)
    
# Splitting dataset
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=.20, random_state=42, shuffle=True)

X_test  = test[columns].to_numpy()
X_train = train[columns].to_numpy()
y_test  = {}
y_train = {}

for col in targets:
    y_test[col]  = test[col].to_numpy()
    y_train[col] = train[col].to_numpy()

## Modeling